In [1]:
## Interactive magics
%matplotlib inline
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import seaborn as sns
import patsy as pt
import pymc3 as pm

plt.rcParams['figure.figsize'] = 14, 6
np.random.seed(0)
print('Running on PyMC3 v{}'.format(pm.__version__))

/Users/marysilva/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Running on PyMC3 v3.5


In [2]:
def strip_derived_rvs(rvs):
    '''Convenience fn: remove PyMC3-generated RVs from a list'''
    ret_rvs = []
    for rv in rvs:
        if not (re.search('_log',rv.name) or re.search('_interval',rv.name)):
            ret_rvs.append(rv)
    return ret_rvs


def plot_traces_pymc(trcs, varnames=None):
    ''' Convenience fn: plot traces with overlaid means and values '''

    nrows = len(trcs.varnames)
    if varnames is not None:
        nrows = len(varnames)

    ax = pm.traceplot(trcs, varnames=varnames, figsize=(12,nrows*1.4),
                      lines={k: v['mean'] for k, v in
                             pm.summary(trcs,varnames=varnames).iterrows()})

    for i, mn in enumerate(pm.summary(trcs, varnames=varnames)['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data',
                         xytext=(5,10), textcoords='offset points', rotation=90,
                         va='bottom', fontsize='large', color='#AA0022')

In [3]:
# decide poisson theta values
theta_noalcohol_meds = 1    # no alcohol, took an antihist
theta_alcohol_meds = 3      # alcohol, took an antihist
theta_noalcohol_nomeds = 6  # no alcohol, no antihist
theta_alcohol_nomeds = 36   # alcohol, no antihist

# create samples
q = 1000
df = pd.DataFrame({
        'nsneeze': np.concatenate((np.random.poisson(theta_noalcohol_meds, q),
                                   np.random.poisson(theta_alcohol_meds, q),
                                   np.random.poisson(theta_noalcohol_nomeds, q),
                                   np.random.poisson(theta_alcohol_nomeds, q))),
        'alcohol': np.concatenate((np.repeat(False, q),
                                   np.repeat(True, q),
                                   np.repeat(False, q),
                                   np.repeat(True, q))),
        'nomeds': np.concatenate((np.repeat(False, q),
                                      np.repeat(False, q),
                                      np.repeat(True, q),
                                      np.repeat(True, q)))})

In [4]:
df

,nsneeze,alcohol,nomeds
0,2,False,False
1,1,False,False
2,1,False,False
3,2,False,False
4,2,False,False
5,1,False,False
6,0,False,False
7,0,False,False
8,5,False,False
9,1,False,False


In [5]:
fml = 'nsneeze ~ alcohol + antihist + alcohol:antihist'  # full patsy formulation
fml = 'nsneeze ~ alcohol * nomeds'  # lazy, alternative patsy formulation
(mx_en, mx_ex) = pt.dmatrices(fml, df, return_type='dataframe', NA_action='raise')

In [6]:
(mx_en, mx_ex)

(      nsneeze
 0         2.0
 1         1.0
 2         1.0
 3         2.0
 4         2.0
 5         1.0
 6         0.0
 7         0.0
 8         5.0
 9         1.0
 10        1.0
 11        2.0
 12        0.0
 13        1.0
 14        1.0
 15        2.0
 16        2.0
 17        1.0
 18        0.0
 19        2.0
 20        0.0
 21        0.0
 22        0.0
 23        1.0
 24        1.0
 25        0.0
 26        0.0
 27        1.0
 28        1.0
 29        0.0
 ...       ...
 3970     41.0
 3971     32.0
 3972     36.0
 3973     46.0
 3974     40.0
 3975     39.0
 3976     40.0
 3977     45.0
 3978     41.0
 3979     29.0
 3980     36.0
 3981     48.0
 3982     39.0
 3983     36.0
 3984     32.0
 3985     40.0
 3986     30.0
 3987     27.0
 3988     31.0
 3989     29.0
 3990     38.0
 3991     31.0
 3992     48.0
 3993     32.0
 3994     33.0
 3995     38.0
 3996     31.0
 3997     30.0
 3998     34.0
 3999     36.0
 
 [4000 rows x 1 columns],       Intercept  alcohol[T.True]  nomeds[T

In [7]:
pd.concat((mx_ex,mx_ex))

,Intercept,alcohol[T.True],nomeds[T.True],alcohol[T.True]:nomeds[T.True]
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0
